In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard imports
import numpy as np
import xarray as xr
import tqdm as tqdm

In [3]:
# Import OpenSense modules as submodules
import sys
import os

sys.path.append(os.path.abspath("./pycomlink/"))
sys.path.append(os.path.abspath("./poligrain/src/"))
sys.path.append(os.path.abspath("./mergeplg/src/"))

import pycomlink as pycml 
import poligrain as plg
import mergeplg 

# OpenMRG

In [4]:
# OpenMRG
ds_rad = xr.open_dataset("data/andersson_2022_OpenMRG/radar/openmrg_rad.nc")                    
ds_cmls = xr.open_dataset("data/processed_cml_OpenMRG.nc").R_acc                                  

# Adjust rainfall fields

In [5]:
os.makedirs('data/adjusted_fields', exist_ok=True)
months = ['2015-06', '2015-07', '2015-08']

In [6]:
# additive IDW
nnear = 12 
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="additive",
        nnear=nnear,
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_IDW": xr.concat(rainfall, dim="time") })
data['rainfall_add_IDW']=xr.where(
    np.isnan(data['rainfall_add_IDW']), 0, data['rainfall_add_IDW']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_add_IDW.nc')    
del data, merger

month: 2015-06


100%|██████████| 2879/2879 [00:41<00:00, 68.84it/s]


month: 2015-07


100%|██████████| 2976/2976 [00:43<00:00, 69.13it/s]


month: 2015-08


100%|██████████| 2976/2976 [00:42<00:00, 69.86it/s]


In [7]:
# multiplicative IDW
nnear = 12 
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="multiplicative",
        nnear=nnear,
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_IDW": xr.concat(rainfall, dim="time") })
data['rainfall_mul_IDW']=xr.where(
    np.isnan(data['rainfall_mul_IDW']), 0, data['rainfall_mul_IDW']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_mul_IDW.nc')    
del data, merger

month: 2015-06


100%|██████████| 2879/2879 [00:41<00:00, 68.96it/s]


month: 2015-07


100%|██████████| 2976/2976 [00:43<00:00, 68.84it/s]


month: 2015-08


100%|██████████| 2976/2976 [00:48<00:00, 61.64it/s]


In [8]:
# additive POINT BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_pBK": xr.concat(rainfall, dim="time") })
data['rainfall_add_pBK']=xr.where(
    np.isnan(data['rainfall_add_pBK']), 0, data['rainfall_add_pBK']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_add_pBK.nc')    
del data, merger

month: 2015-06


100%|██████████| 2879/2879 [01:19<00:00, 36.15it/s]


month: 2015-07


100%|██████████| 2976/2976 [01:29<00:00, 33.31it/s]


month: 2015-08


100%|██████████| 2976/2976 [01:17<00:00, 38.57it/s]


In [9]:
# multiplicative POINT BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_pBK": xr.concat(rainfall, dim="time") })
data['rainfall_mul_pBK']=xr.where(
    np.isnan(data['rainfall_mul_pBK']), 0, data['rainfall_mul_pBK']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_mul_pBK.nc')    
del data, merger

month: 2015-06


100%|██████████| 2879/2879 [01:10<00:00, 41.10it/s]


month: 2015-07


100%|██████████| 2976/2976 [01:19<00:00, 37.55it/s]


month: 2015-08


100%|██████████| 2976/2976 [01:10<00:00, 42.17it/s]


In [10]:
# additive LINE BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_lBK": xr.concat(rainfall, dim="time") })
data['rainfall_add_lBK']=xr.where(
    np.isnan(data['rainfall_add_lBK']), 0, data['rainfall_add_lBK']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_add_lBK.nc')    
del data, merger

month: 2015-06


100%|██████████| 2879/2879 [01:19<00:00, 36.06it/s]


month: 2015-07


100%|██████████| 2976/2976 [01:29<00:00, 33.15it/s]


month: 2015-08


100%|██████████| 2976/2976 [01:18<00:00, 37.88it/s]


In [11]:
# multiplicative LINE BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_lBK": xr.concat(rainfall, dim="time") })
data['rainfall_mul_lBK']=xr.where(
    np.isnan(data['rainfall_mul_lBK']), 0, data['rainfall_mul_lBK']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_mul_lBK.nc')    
del data, merger

month: 2015-06


100%|██████████| 2879/2879 [01:16<00:00, 37.83it/s]


month: 2015-07


100%|██████████| 2976/2976 [01:24<00:00, 35.07it/s]


month: 2015-08


100%|██████████| 2976/2976 [01:09<00:00, 42.70it/s]


In [12]:
# KED
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_ked": xr.concat(rainfall, dim="time") })
data['rainfall_ked']=xr.where(
    np.isnan(data['rainfall_ked']), 0, data['rainfall_ked']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_ked.nc')    
del data, merger

month: 2015-06


100%|██████████| 2879/2879 [01:13<00:00, 39.06it/s]


month: 2015-07


100%|██████████| 2976/2976 [01:29<00:00, 33.44it/s]


month: 2015-08


100%|██████████| 2976/2976 [01:13<00:00, 40.41it/s]


# OpenRainER

In [13]:
# OpenRainER
ds_rad = xr.open_dataset("data/covi_2024_OpenRainER/openrainer_radar.nc")         
ds_cmls = xr.open_dataset("data/processed_cml_OpenRainER.nc").R_acc                     

# Adjust rainfall fields

In [14]:
os.makedirs('data/adjusted_fields', exist_ok=True)
months = ['2022-06', '2022-07', '2022-08']

In [15]:
# additive IDW
nnear = 12 
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="additive",
        nnear=nnear,
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_IDW": xr.concat(rainfall, dim="time") })
data['rainfall_add_IDW']=xr.where(
    np.isnan(data['rainfall_add_IDW']), 0, data['rainfall_add_IDW']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_add_IDW.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [03:10<00:00, 15.11it/s]


month: 2022-07


100%|██████████| 2976/2976 [03:14<00:00, 15.31it/s]


month: 2022-08


100%|██████████| 2976/2976 [03:25<00:00, 14.49it/s]


In [16]:
# multiplicative IDW
nnear = 12 
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="multiplicative",
        nnear=nnear,
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_IDW": xr.concat(rainfall, dim="time") })
data['rainfall_mul_IDW']=xr.where(
    np.isnan(data['rainfall_mul_IDW']), 0, data['rainfall_mul_IDW']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_mul_IDW.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [02:52<00:00, 16.72it/s]


month: 2022-07


100%|██████████| 2976/2976 [03:06<00:00, 15.96it/s]


month: 2022-08


100%|██████████| 2976/2976 [03:15<00:00, 15.19it/s]


In [17]:
# multiplicative IDW filtered
nnear = 12 
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="multiplicative",
        nnear=nnear,
        multiplicative_factors=(0.1, 2),
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_IDW": xr.concat(rainfall, dim="time") })
data['rainfall_mul_IDW']=xr.where(
    np.isnan(data['rainfall_mul_IDW']), 0, data['rainfall_mul_IDW']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_mul_IDW2.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [02:48<00:00, 17.05it/s]


month: 2022-07


100%|██████████| 2976/2976 [02:52<00:00, 17.24it/s]


month: 2022-08


100%|██████████| 2976/2976 [03:18<00:00, 14.99it/s]


In [18]:
# additive POINT BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_pBK": xr.concat(rainfall, dim="time") })
data['rainfall_add_pBK']=xr.where(
    np.isnan(data['rainfall_add_pBK']), 0, data['rainfall_add_pBK']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_add_pBK.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [24:02<00:00,  2.00it/s] 


month: 2022-07


100%|██████████| 2976/2976 [07:32<00:00,  6.57it/s]


month: 2022-08


100%|██████████| 2976/2976 [19:09<00:00,  2.59it/s]


In [19]:
# multiplicative POINT BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_pBK": xr.concat(rainfall, dim="time") })
data['rainfall_mul_pBK']=xr.where(
    np.isnan(data['rainfall_mul_pBK']), 0, data['rainfall_mul_pBK']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_mul_pBK.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [14:21<00:00,  3.34it/s] 


month: 2022-07


100%|██████████| 2976/2976 [05:40<00:00,  8.75it/s]


month: 2022-08


100%|██████████| 2976/2976 [14:14<00:00,  3.48it/s]


In [20]:
# multiplicative POINT BLOCK KRIGING filtered
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        multiplicative_factors=(0.1, 2),
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_pBK": xr.concat(rainfall, dim="time") })
data['rainfall_mul_pBK']=xr.where(
    np.isnan(data['rainfall_mul_pBK']), 0, data['rainfall_mul_pBK']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_mul_pBK2.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [12:54<00:00,  3.72it/s] 


month: 2022-07


100%|██████████| 2976/2976 [05:35<00:00,  8.87it/s]


month: 2022-08


100%|██████████| 2976/2976 [13:26<00:00,  3.69it/s]


In [21]:
# additive LINE BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_lBK": xr.concat(rainfall, dim="time") })
data['rainfall_add_lBK']=xr.where(
    np.isnan(data['rainfall_add_lBK']), 0, data['rainfall_add_lBK']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_add_lBK.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [24:11<00:00,  1.98it/s] 


month: 2022-07


100%|██████████| 2976/2976 [07:34<00:00,  6.54it/s]


month: 2022-08


100%|██████████| 2976/2976 [19:09<00:00,  2.59it/s]


In [22]:
# multiplicative LINE BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_lBK": xr.concat(rainfall, dim="time") })
data['rainfall_mul_lBK']=xr.where(
    np.isnan(data['rainfall_mul_lBK']), 0, data['rainfall_mul_lBK']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_mul_lBK.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [14:22<00:00,  3.34it/s] 


month: 2022-07


100%|██████████| 2976/2976 [05:39<00:00,  8.75it/s]


month: 2022-08


100%|██████████| 2976/2976 [14:18<00:00,  3.47it/s]


In [23]:
# multiplicative LINE BLOCK KRIGING filtered
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        multiplicative_factors=(0.1, 2),
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_lBK": xr.concat(rainfall, dim="time") })
data['rainfall_mul_lBK']=xr.where(
    np.isnan(data['rainfall_mul_lBK']), 0, data['rainfall_mul_lBK']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_mul_lBK2.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [12:56<00:00,  3.71it/s] 


month: 2022-07


100%|██████████| 2976/2976 [05:47<00:00,  8.56it/s]


month: 2022-08


100%|██████████| 2976/2976 [13:34<00:00,  3.65it/s]


In [24]:
# KED
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_ked": xr.concat(rainfall, dim="time") })
data['rainfall_ked']=xr.where(
    np.isnan(data['rainfall_ked']), 0, data['rainfall_ked']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_ked.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [21:15<00:00,  2.26it/s] 


month: 2022-07


100%|██████████| 2976/2976 [04:56<00:00, 10.05it/s] 


month: 2022-08


100%|██████████| 2976/2976 [16:26<00:00,  3.02it/s] 
